In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bredasdorp,ZA,2021-01-31 00:36:00,-34.5322,20.0403,66.99,84,0,6.22
1,1,Saint George,US,2021-01-31 00:31:13,37.1041,-113.5841,50.00,50,1,1.41
2,2,Georgetown,MY,2021-01-31 00:36:06,5.4112,100.3354,77.00,83,20,6.78
3,3,Ushuaia,AR,2021-01-31 00:35:51,-54.8000,-68.3000,51.80,62,20,21.85
4,4,Vardo,NO,2021-01-31 00:36:11,70.3705,31.1107,25.18,95,100,19.01


In [3]:
# Get the data types.
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
print(len(temps))

566


In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating = True, max_intensity = 300, point_radius = 15)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 15
What is the maximum temperature you would like for your trip? 30


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Vardo,NO,2021-01-31 00:36:11,70.3705,31.1107,25.18,95,100,19.01
6,6,Torbay,CA,2021-01-31 00:36:12,47.6666,-52.7314,28.99,93,90,8.05
34,34,Aksu,CN,2021-01-31 00:36:19,41.1231,80.2644,15.80,86,28,3.02
43,43,Jumla,NP,2021-01-31 00:36:22,29.2747,82.1838,29.57,56,0,3.62
64,64,Bonaventure,CA,2021-01-31 00:36:27,48.0457,-65.4926,15.80,62,98,10.36
66,66,Havre-Saint-Pierre,CA,2021-01-31 00:36:28,50.2334,-63.5986,19.40,73,90,11.50
68,68,Jamestown,US,2021-01-31 00:33:08,42.0970,-79.2353,17.60,86,1,3.44
81,81,Olafsvik,IS,2021-01-31 00:36:22,64.8945,-23.7142,23.16,89,30,16.17
92,92,Port-Cartier,CA,2021-01-31 00:32:06,50.0334,-66.8654,19.40,63,19,6.91
96,96,Zirovnice,CZ,2021-01-31 00:36:33,49.2532,15.1882,24.01,89,100,1.01


In [10]:
preferred_cities_df.count()

City_ID       79
City          79
Country       79
Date          79
Lat           79
Lng           79
Max Temp      79
Humidity      79
Cloudiness    79
Wind Speed    79
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Vardo,NO,25.18,70.3705,31.1107,
6,Torbay,CA,28.99,47.6666,-52.7314,
34,Aksu,CN,15.80,41.1231,80.2644,
43,Jumla,NP,29.57,29.2747,82.1838,
64,Bonaventure,CA,15.80,48.0457,-65.4926,
...,...,...,...,...,...,...
542,Mullovka,RU,24.80,54.2000,49.4000,
546,Marfino,RU,26.01,55.7028,37.3822,
549,Usogorsk,RU,24.31,63.4106,48.6872,
563,Havoysund,NO,26.60,70.9963,24.6622,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [13]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Vardo,NO,25.18,70.3705,31.1107,Vardø Hotel AS
6,Torbay,CA,28.99,47.6666,-52.7314,Gallows Cove
34,Aksu,CN,15.80,41.1231,80.2644,Pudong Holiday Hotel
43,Jumla,NP,29.57,29.2747,82.1838,"अन्नी घर, जुम्ला - Anni Ghar"
64,Bonaventure,CA,15.80,48.0457,-65.4926,Riôtel Bonaventure (N° établissement : 1271)
...,...,...,...,...,...,...
542,Mullovka,RU,24.80,54.2000,49.4000,Tualet
546,Marfino,RU,26.01,55.7028,37.3822,ARK guest house
549,Usogorsk,RU,24.31,63.4106,48.6872,"""Dom Invest - Kvartiry Posutochno"""
563,Havoysund,NO,26.60,70.9963,24.6622,Arctic View Glamping & Restaurant


In [15]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [17]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [18]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))